## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
sys.path.append('../')

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Victor Harbor,AU,-35.5667,138.6167,62.01,73,22,1.01,few clouds
1,1,Sobolevo,RU,54.4299,31.8996,22.28,94,100,9.66,snow
2,2,Yenisehir,TR,40.2644,29.6531,39.25,81,75,6.91,light intensity shower rain
3,3,Ponta Do Sol,PT,32.6667,-17.1000,70.12,70,94,9.46,overcast clouds
4,4,Northam,GB,51.0333,-4.2167,51.42,93,85,15.77,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Victor Harbor,AU,-35.5667,138.6167,62.01,73,22,1.01,few clouds
3,3,Ponta Do Sol,PT,32.6667,-17.1000,70.12,70,94,9.46,overcast clouds
10,10,Bengkulu,ID,-3.8004,102.2655,76.23,91,100,5.97,overcast clouds
13,13,Merauke,ID,-8.4667,140.3333,79.02,82,89,11.07,overcast clouds
14,14,Salalah,OM,17.0151,54.0924,77.09,69,0,3.44,clear sky
18,18,Busselton,AU,-33.6500,115.3333,75.97,41,82,9.08,broken clouds
19,19,Caravelas,BR,-17.7125,-39.2481,85.24,59,47,9.57,scattered clouds
20,20,Cape Town,ZA,-33.9258,18.4232,73.81,51,20,4.00,few clouds
22,22,Avarua,CK,-21.2078,-159.7750,78.85,83,75,14.97,broken clouds
23,23,Bela,IN,25.9333,81.9833,61.25,76,51,17.00,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Victor Harbor,AU,62.01,few clouds,-35.5667,138.6167,
3,Ponta Do Sol,PT,70.12,overcast clouds,32.6667,-17.1000,
10,Bengkulu,ID,76.23,overcast clouds,-3.8004,102.2655,
13,Merauke,ID,79.02,overcast clouds,-8.4667,140.3333,
14,Salalah,OM,77.09,clear sky,17.0151,54.0924,
18,Busselton,AU,75.97,broken clouds,-33.6500,115.3333,
19,Caravelas,BR,85.24,scattered clouds,-17.7125,-39.2481,
20,Cape Town,ZA,73.81,few clouds,-33.9258,18.4232,
22,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,
23,Bela,IN,61.25,broken clouds,25.9333,81.9833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Victor Harbor,AU,62.01,few clouds,-35.5667,138.6167,McCracken Country Club
3,Ponta Do Sol,PT,70.12,overcast clouds,32.6667,-17.1000,Hotel do Campo
10,Bengkulu,ID,76.23,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
13,Merauke,ID,79.02,overcast clouds,-8.4667,140.3333,
14,Salalah,OM,77.09,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
18,Busselton,AU,75.97,broken clouds,-33.6500,115.3333,Observatory Guest House
19,Caravelas,BR,85.24,scattered clouds,-17.7125,-39.2481,Pousada dos Navegantes
20,Cape Town,ZA,73.81,few clouds,-33.9258,18.4232,Southern Sun Waterfront Cape Town
22,Avarua,CK,78.85,broken clouds,-21.2078,-159.7750,Paradise Inn
23,Bela,IN,61.25,broken clouds,25.9333,81.9833,Raj Bhawan


In [10]:
import numpy as np

In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Victor Harbor,AU,62.01,few clouds,-35.5667,138.6167,McCracken Country Club
3,Ponta Do Sol,PT,70.12,overcast clouds,32.6667,-17.1000,Hotel do Campo
10,Bengkulu,ID,76.23,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
13,Merauke,ID,79.02,overcast clouds,-8.4667,140.3333,NaN
14,Salalah,OM,77.09,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
...,...,...,...,...,...,...,...
678,Dakar,SN,92.28,clear sky,14.6937,-17.4441,Redface Studios Meublés
682,Port-Gentil,GA,80.24,scattered clouds,-0.7193,8.7815,Fengshui Residence
683,Beloha,MG,74.26,clear sky,-25.1667,45.0500,NaN
688,Kolondieba,ML,92.37,clear sky,11.0882,-6.8926,Hotel Dakan


In [12]:
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Victor Harbor,AU,62.01,few clouds,-35.5667,138.6167,McCracken Country Club
3,Ponta Do Sol,PT,70.12,overcast clouds,32.6667,-17.1000,Hotel do Campo
10,Bengkulu,ID,76.23,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
14,Salalah,OM,77.09,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
18,Busselton,AU,75.97,broken clouds,-33.6500,115.3333,Observatory Guest House


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))